### 18.1 search a maze
Given a 2D array of black and white entries representing a maze with designated entrance and exit points, find a path from the entrance to the exit, if one exists.

In [1]:
# Run this in Python once, it should take effect permanently
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

{'CodeCell': {'cm_config': {'autoCloseBrackets': False}},
 'load_extensions': {'contrib_nbextensions_help_item/main': True,
  'nbextensions_configurator/config_menu/main': True,
  'varInspector/main': True,
  'vim_binding/vim_binding': True}}

In [2]:
import collections
import random
import sys

from termcolor import colored

Coordinate = collections.namedtuple('Coordinate', ('x', 'y'))
ROAD, WALL = range(2)

def search_maze(maze, s, e):
    # Perform DFS to find a feasible path.
    def search_maze_helper(cur):
        # Checks cur is within maze boundaries and is a path pixel.
        if not (0 <= cur.x < len(maze) and 0 <= cur.y < len(maze[cur.x]) and
                maze[cur.x][cur.y] == ROAD):
            return False
        path.append(cur)
        maze[cur.x][cur.y] = WALL
        if cur == e:
            return True

        # go in all four directions, (recursive step)
        if any(
                map(search_maze_helper, (
                    Coordinate(cur.x - 1, cur.y), Coordinate( cur.x + 1, cur.y),
                    Coordinate(cur.x, cur.y - 1), Coordinate(cur.x, cur.y + 1)))):
            return True
       
        # you have no valid moves
        # delete yourself from the path
        del path[-1]
        # return False to show that this is not a good path 
        return False

    # initialize with an empty path
    path = []
    # use the recursive search helper to find a path
    if not search_maze_helper(s):
        return []  # No path between s and e.
    return path


def print_maze(maze):
    for line in maze:
        line = " ".join([str(i) for i in line])
        print(line)

def print_solved_maze(maze, start, end, path):
    if not path:
        print("There is no path from start to end")
    else:
        print("There is a path from start to end")

    while path:
        coordinate = path.pop()
        maze[coordinate.x][coordinate.y] = colored("x", "yellow")

    maze[start.x][start.y] = colored("s", "green")
    maze[end.x][end.y] = colored("e", "red")
    
    print()
    print_maze(maze)
    
n = m = 10 
# Produce an unsolved n x m maze
maze = [[random.randrange(2) for _ in range(m)] for _ in range(n)]

# get a list of road pixels (black is the wall)
road = []
for i in range(n):
    for j in range(m):
        if maze[i][j] == ROAD:
            road.append(Coordinate(i, j))

# pick two random points on the list of road, call them start and finish            
if road:
    start = random.randrange(len(road))
    end = random.randrange(len(road))


print("Starting: {}, Ending: {}".format(road[start], road[end]))
path = search_maze(maze, road[start], road[end])
print_solved_maze(maze, road[start], road[end], path)


Starting: Coordinate(x=9, y=4), Ending: Coordinate(x=1, y=8)
There is a path from start to end

0 0 0 0 1 1 1 x x 1
1 0 0 0 1 1 1 x e 1
1 0 0 1 1 1 x x 0 0
0 1 0 1 1 1 x 1 1 0
0 0 1 1 1 0 x 1 0 0
0 1 1 0 0 1 x 1 0 0
1 0 0 1 0 1 x 1 1 1
1 1 1 1 1 x x 0 0 0
1 1 0 1 x x 0 1 1 0
0 0 1 0 s 1 1 1 1 0


### Remarks:

This seems to work well enough and is a depth-first search.  It's interesting that you can create a class for coordinates so easily.  The search_maze_helper is a recursive method that first ensures that the coordinate is within the maze and that it is *WHITE* (empty space),  Next, it adds that to the *path* list and turns the coordinate *BLACK* so that it doesn't create a circular path.  Then, it calls all four possible moves (up, down, left, and right) recursively on the updated map and if **any** of them are good it keeps going.  At some point one of the recursively called **search_maze_helper** may return **True**, meaning that **s** and **e** are the same thing and we have found a path.  Otherwise, there is no path.